In [1]:
import numpy as np
from scipy.spatial import voronoi_plot_2d, Voronoi, Delaunay, delaunay_plot_2d
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib qt
import sys
sys.path.append("/home/remi/Macular-Oxygenation/Oxygen-Voxel-Method/src/")
from VascularNetwork import VascularNetwork as VN
CreateGraph = VN.CreateGraph

In [2]:
def GraphToCCO(CCOFileName : str, G : nx.DiGraph, **kwargs):
    
    # Various tree info in the header of the CCO file
    qProx = kwargs.get('qProx', 2.5000000000000001e-04) # 15 microliter/min in cm3/s, probably
    psiFactor = kwargs.get('psiFactor', 9.6040000000001100e-06) # Not sure what that does
    dp = kwargs.get('dp', 9.5453193691098149e+03) # Pressure drop from the single inlet to all outlet vessels, in Pa
    refPressure = kwargs.get('refPressure', 6.6661199999999999e+03) # Occular perfusion pressure, 50mmHg in Pa
    nTerms = 2

    radiusCRA = G[0][1]['radius']

    with open(CCOFileName, 'w') as f:
        f.write('*Tree\n')
        x = G.nodes[0]['position']
        f.write(f"{x[0]} {x[1]} {x[2]} {qProx} {psiFactor} {dp} {nTerms} {refPressure} {G.number_of_nodes()} {radiusCRA} {1e-6}\n")

        f.write('\n*Vessels\n')
        f.write(f"{G.number_of_edges()}\n")
        for n1, n2, data in G.edges(data=True):
            xProx, xDist = G.nodes[n1]['position'], G.nodes[n2]['position']
            branchingMode = 2 if data['key']==0 else 2 # If the root, then bifurcates at distal end only, else deformable parent
            vesselFunction = 0 # Distribution vessel, bifurcates in the domain only
            f.write(f"{data['key']} {xProx[0]} {xProx[1]} {xProx[2]} {xDist[0]} {xDist[1]} {xDist[2]}")
            f.write(f"0.0 0.0 0.0 0.0 {branchingMode} {data['radius']} {qProx} 0.0 0.0 0.0 {vesselFunction} 0.0 0.0 -1\n")

        f.write('\n*Connectivity')
        for n1, n2, data in G.edges(data=True):
            f.write('\n')

            # Sanity check
            parent = G.predecessors(n1)
            assert sum(1 for _ in parent) <= 1, f'Oops... Something went wrong, more than 1 parent was found {parent=}.'
            descendents = G.successors(n2) 
            assert sum(1 for _ in descendents) <= 2, f'Oops... Something went wrong, more than 2 descendents were found {descendents=}.' 
            # End sanity check 

            parent = G.predecessors(n1)
            descendents = G.successors(n2) 
            try: 
                parentKey = next(parent)
            except:
                parentKey = -1
            f.write(f"{data['key']} {parentKey}")                    
            
            for descendent in descendents:
                f.write(f" {G[n2][descendent]['key']}")

In [3]:
G = CreateGraph('../Results-8-patients/Patient1/sim_1.cco', 'mm')

lengthConversion=10.0
The tree has 4015 vessels.


In [4]:
for i,e in enumerate(G.edges()):
    nParents = 0
    currentEdge = e
    while True and nParents < 6:
        try:
            currentEdge=next(G.predecessors(e[0]))
            nParents+=1
        except StopIteration:
            break
    nx.set_edge_attributes(G, {e:{'level':nParents}})

print(max([r for _,_,r in G.edges.data('radius')]))


0.021358677843486894


In [5]:
depth = np.mean([pos[-1] for n,pos in G.nodes.data('position')])-0.041 # mm
newEdges = []
newNodes = []
removeEdges = []
n = G.number_of_nodes()-1
for n1,n2,r in G.edges.data('radius'):
    if r<0.01: # Only larger arterioles dive to ICP
        continue

    r = np.random.random()
    if r<0.2:
        midPoint = (G.nodes[n1]['position']+G.nodes[n2]['position'])/2
        level,l = G[n1][n2]['level'], G[n1][n2]['length']/2.0
        
        n+=1
        newNodes.append((n, {'position':midPoint}))
        newEdges.append((n1,n, {'radius':r, 'length':l, 'level':level}))
        newEdges.append((n,n2,{'radius':r, 'length':l, 'level':level}))
        removeEdges.append((n1,n2))
        
        n+=1
        midPoint[2] = depth
        newNodes.append((n,{'position':midPoint}))
        newEdges.append((n-1,n,{'radius':r, 'length':depth, 'level':level+1, 'type':'SVP-ICP'}))

print(f"Creating {len(removeEdges)} connections.")

G.add_nodes_from(newNodes)
G.add_edges_from(newEdges)        
G.remove_edges_from(removeEdges)

Creating 67 connections.


In [6]:
nx.draw_networkx_nodes(G, pos={n:pos[:2] for n,pos in G.nodes(data='position')},
                       nodelist=[n[0] for n in newNodes[::2]], node_size=10)

In [7]:
points = np.array([pos[:2] for n,pos in G.nodes.data('position') if pos[-1]<0 ])
vor = Voronoi(points)
fig = voronoi_plot_2d(vor)
plt.show()

In [8]:
points = np.array([pos[:2] for n,pos in G.nodes.data('position') if pos[-1]<0 ])
delau = Delaunay(points)
fig = delaunay_plot_2d(delau)
plt.show()

In [9]:
### Compute Voronoi from Delaunay triangulation (e.g., center of the triangles circumcircle)
p = delau.points[delau.vertices]
# Triangle vertices
A = p[:,0,:].T
B = p[:,1,:].T
C = p[:,2,:].T

# See http://en.wikipedia.org/wiki/Circumscribed_circle#Circumscribed_circles_of_triangles
# The following is just a direct transcription of the formula there
a = A - C
b = B - C

def dot2(u, v):
    return u[0]*v[0] + u[1]*v[1]

def cross2(u, v, w):
    """u x (v x w)"""
    return dot2(u, w)*v - dot2(u, v)*w

def ncross2(u, v):
    """|| u x v ||^2"""
    return sq2(u)*sq2(v) - dot2(u, v)**2

def sq2(u):
    return dot2(u, u)

cc = cross2(sq2(a) * b - sq2(b) * a, a, b) / (2*ncross2(a, b)) + C
# Grab the Voronoi edges
vc = cc[:,delau.neighbors]
vc[:,delau.neighbors == -1] = np.nan # edges at infinity, plotting those would need more work...

lines = []
lines.extend(zip(cc.T, vc[:,:,0].T))
lines.extend(zip(cc.T, vc[:,:,1].T))
lines.extend(zip(cc.T, vc[:,:,2].T))

from matplotlib.collections import LineCollection
lines = LineCollection(lines, edgecolor='k')

fig = delaunay_plot_2d(delau)
ax = fig.get_axes()
ax[0].scatter(*cc, c='red')
plt.gca().add_collection(lines)

In [10]:
radiusFAZ_ICP = 0.4 # mm
#penetratingArterioles = np.array([pos[:2] for n,pos in G.nodes.data('position') if pos[-1]<0 ])
penetratingArterioles = np.array([n[1]['position'][:2] for n in newNodes[::2]])
theta = np.random.random(50) * np.pi*2
fazContour = np.array([radiusFAZ_ICP*np.cos(theta), radiusFAZ_ICP*np.sin(theta)]).T

vor = Voronoi(penetratingArterioles, incremental=True)
vor.add_points(fazContour)
fig = voronoi_plot_2d(vor)
plt.show()

In [11]:
newVertices = vor.vertices[np.logical_and(np.linalg.norm(vor.vertices, axis=1)<2.0, np.linalg.norm(vor.vertices, axis=1)>radiusFAZ_ICP)]

fig, ax = plt.subplots()
ax.scatter(*newVertices.T, s=5)
ax.scatter(*points.T, c='k', s=5)

lines = vor.points[vor.ridge_points]
availableConnections = newVertices
otherLines = np.array([[p,newVertices[np.argmin(np.linalg.norm(p[:2]-newVertices, axis=1))]] 
                         for p in penetratingArterioles])

lines = np.dstack((lines.T, otherLines.T)).T

lines = LineCollection(lines)
ax.add_collection(lines)

In [12]:
connectingVeins = np.random.random(size=penetratingArterioles.shape)*3.0-1.5
plt.scatter(*penetratingArterioles.T)
plt.scatter(*connectingVeins.T)

plt.legend(['Connecting arterioles', 'Connecting veins'])

In [13]:
ICP = nx.Graph()
ICP.add_nodes_from([(n, {'position':np.append(vor.points[n][:2],depth), 'type':'art'}) for n in vor.ridge_points.ravel() if np.linalg.norm(vor.points[n])>=0*radiusFAZ_ICP])
ICP.add_nodes_from([(n, {'position':np.append(vor.vertices[n][:2],depth), 'type':'cap'}) for n in np.array(vor.ridge_vertices).ravel() if np.linalg.norm(vor.vertices[n][:2])<1.5])
ICP.add_nodes_from([(f'v{i}', {'position':np.append(v[:2],depth), 'type':'vei'}) for i,v in enumerate(connectingVeins)])

fig, ax = plt.subplots(figsize=(9,6))
nx.draw_networkx(ICP, pos={n:pos[:2] for n,pos in ICP.nodes(data='position')}, 
                 node_color=['r' if t=='art' else 'b' if t=='cap' else 'g' for n,t in ICP.nodes.data('type')],
                 node_size=10, with_labels=False, ax=ax)

import matplotlib.patches as mpatches
plt.legend(handles=[mpatches.Patch(color='r', label='Connecting arterioles'),
                    mpatches.Patch(color='b', label='Capillary nodes'),
                    mpatches.Patch(color='g', label='Connecting venules')])

In [14]:
ICP.add_edges_from((i[0],i[1]) for i in vor.ridge_points if (ICP.has_node(i[0]) and ICP.has_node(i[1])))
fig, ax = plt.subplots(figsize=(9,6))
nx.draw_networkx(ICP, pos={n:pos[:2] for n,pos in ICP.nodes(data='position', default=np.array([-2,-2]))}, 
                 node_color=['r' if t=='art' else 'b' if t=='cap' else 'g' if t=='vei' else 'k' for n,t in ICP.nodes.data('type')],
                 node_size=10, with_labels=False, ax=ax)

import matplotlib.patches as mpatches
plt.legend(handles=[mpatches.Patch(color='r', label='Connecting arterioles'),
                    mpatches.Patch(color='b', label='Capillary nodes'),
                    mpatches.Patch(color='g', label='Connecting venules')])

In [15]:
print(vor.ridge_points[0], ICP[0])

[ 3 13] {3: {}, 9: {}, 10: {}, 38: {}, 39: {}, 1: {}}


In [16]:
n = 30
theta = np.random.random(n)*np.pi*2
r = np.sqrt(np.random.random(n)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
points = np.array([r*np.cos(theta), r*np.sin(theta)]).T
vor = Voronoi(points)
fig = voronoi_plot_2d(vor)

In [17]:
ICP = nx.Graph()

pointsID = np.unique(vor.ridge_points)
vei = pointsID.tolist()
art = [vei.pop(np.random.randint(0, len(vei))) for _ in range(int(len(pointsID)/2))]

ICP.add_nodes_from([(n, {'position':vor.points[n], 'type':'art'}) for n in art])
ICP.add_nodes_from([(n, {'position':vor.points[n], 'type':'vei'}) for n in vei])
ICP.add_nodes_from([(n, {'position':vor.vertices[n], 'type':'cap'}) 
                     for n in np.array(vor.ridge_vertices).ravel() 
                     if (np.linalg.norm(vor.vertices[n])<2 and not ICP.has_node(n))])

count = {'art':0, 'cap':0, 'vei':0}
for n,t in ICP.nodes.data('type'):
    count[t]+=1
print(count)

nx.draw_networkx(ICP, pos={n:pos[:2] for n,pos in ICP.nodes(data='position')}, 
                 node_color=['r' if t=='art' else 'b' if t=='cap' else 'g' if t=='vei' else 'k' for n,t in ICP.nodes.data('type')],
                 node_size=10, with_labels=False)

import matplotlib.patches as mpatches
plt.legend(handles=[mpatches.Patch(color='r', label='Connecting arterioles'),
                    mpatches.Patch(color='b', label='Capillary nodes'),
                    mpatches.Patch(color='g', label='Connecting venules')])


{'art': 15, 'cap': 17, 'vei': 15}


In [18]:
capPos = np.array([np.append(data['position'],n) for n,data in ICP.nodes(data=True) if data['type']=='cap'])
edges = []
for n,data in ICP.nodes.data(True):
    if data['type'] == 'cap':
        continue
    pos = data['position']
    closestCapId = np.argmin(np.linalg.norm(capPos[:,:2]-pos, axis=1))

    if data['type']=='art':
        edges.append((n, capPos[closestCapId,-1]))
    else:
        edges.append((capPos[closestCapId,-1], n))

    # capPos = np.delete(capPos, closestCapId, axis=0)   

ICP.add_edges_from(edges)

nx.draw_networkx(ICP, pos={n:pos[:2] for n,pos in ICP.nodes(data='position')}, 
                 node_color=['r' if t=='art' else 'b' if t=='cap' else 'g' if t=='vei' else 'k' for n,t in ICP.nodes.data('type')],
                 node_size=10, with_labels=False)

import matplotlib.patches as mpatches
plt.legend(handles=[mpatches.Patch(color='r', label='Connecting arterioles'),
                    mpatches.Patch(color='b', label='Capillary nodes'),
                    mpatches.Patch(color='g', label='Connecting venules')])


In [19]:
n = 5
r = np.sqrt(np.random.random(n)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
theta = np.random.random(n)*np.pi*2
art = np.array([r*np.cos(theta), r*np.sin(theta)]).T

r = np.sqrt(np.random.random(n)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
theta = np.random.random(n)*np.pi*2
vei = np.array([r*np.cos(theta), r*np.sin(theta)]).T

tri = Delaunay(np.vstack((art, vei)))
fig = delaunay_plot_2d(tri)

In [20]:
def compute_triangle_circumcenters(xy_pts, tri_arr):
    """
    Compute the centers of the circumscribing circle of each triangle in a triangulation.
    :param np.array xy_pts : points array of shape (n, 2)
    :param np.array tri_arr : triangles array of shape (m, 3), each row is a triple of indices in the xy_pts array

    :return: circumcenter points array of shape (m, 2)
    """
    tri_pts = xy_pts[tri_arr]  # (m, 3, 2) - triangles as points (not indices)

    # finding the circumcenter (x, y) of a triangle defined by three points:
    # (x-x0)**2 + (y-y0)**2 = (x-x1)**2 + (y-y1)**2
    # (x-x0)**2 + (y-y0)**2 = (x-x2)**2 + (y-y2)**2
    #
    # becomes two linear equations (squares are canceled):
    # 2(x1-x0)*x + 2(y1-y0)*y = (x1**2 + y1**2) - (x0**2 + y0**2)
    # 2(x2-x0)*x + 2(y2-y0)*y = (x2**2 + y2**2) - (x0**2 + y0**2)
    a = 2 * (tri_pts[:, 1, 0] - tri_pts[:, 0, 0])
    b = 2 * (tri_pts[:, 1, 1] - tri_pts[:, 0, 1])
    c = 2 * (tri_pts[:, 2, 0] - tri_pts[:, 0, 0])
    d = 2 * (tri_pts[:, 2, 1] - tri_pts[:, 0, 1])

    v1 = (tri_pts[:, 1, 0] ** 2 + tri_pts[:, 1, 1] ** 2) - (tri_pts[:, 0, 0] ** 2 + tri_pts[:, 0, 1] ** 2)
    v2 = (tri_pts[:, 2, 0] ** 2 + tri_pts[:, 2, 1] ** 2) - (tri_pts[:, 0, 0] ** 2 + tri_pts[:, 0, 1] ** 2)

    # solve 2x2 system (see https://en.wikipedia.org/wiki/Invertible_matrix#Inversion_of_2_%C3%97_2_matrices)
    det = (a * d - b * c)
    detx = (v1 * d - v2 * b)
    dety = (a * v2 - c * v1)

    x = detx / det
    y = dety / det

    return (np.vstack((x, y))).T

In [21]:
# n = 100
# theta = np.random.random(n+1)*np.pi*2
# r = np.sqrt(np.random.random(n+1)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
# r[-1] = 0 
# points = np.array([r*np.cos(theta), r*np.sin(theta)]).T
ntheta, nr = 30, 10
theta = np.linspace(0, np.pi*2, ntheta)
r = np.sqrt(np.linspace(0,1, nr)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
u,v = np.meshgrid(theta, r)
u = u.flatten()
v = v.flatten()
x=v*np.cos(u)
y=v*np.sin(u)
points = np.vstack((np.vstack([x,y]).T, (0,0)))
tri = Delaunay(points)
fig = delaunay_plot_2d(tri)

In [114]:
ICP = nx.Graph()
for path in tri.simplices:
    nx.add_path(ICP, path, radius=5e-3)
nx.set_node_attributes(ICP, {n:
                             {'position':points[n], 'type':'cap'} 
                             for n in ICP.nodes()})
for n, pos in ICP.nodes.data('position'):
    if np.allclose(pos,np.zeros(2)):
        print(n, pos, ICP.number_of_nodes())
        ICP.remove_node(n)
        break
#ICP.remove_node(ICP.number_of_nodes()-1) # delete the (0,0) node

def PlotGraphICP(graph):
    fig, ax = plt.subplots(figsize=(9,6))
    nx.draw_networkx(graph, pos={n:pos[:2] for n,pos in graph.nodes(data='position', default=np.array([-2,-2]))}, 
                    node_color=['r' if t=='art' else 'b' if t=='cap' else 'g' if t=='vei' else 'k' for n,t in graph.nodes.data('type')],
                    node_size=10, with_labels=False, ax=ax)

    import matplotlib.patches as mpatches
    plt.legend(handles=[mpatches.Patch(color='r', label='Connecting arterioles'),
                        mpatches.Patch(color='b', label='Capillary nodes'),
                        mpatches.Patch(color='g', label='Connecting venules')])
    plt.show()

PlotGraphICP(ICP)

300 [0. 0.] 291


In [115]:
## The connecting points
n = 10
theta = np.random.random(n)*np.pi*2
#r = np.sqrt(np.random.random(n)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
r = np.sqrt(np.random.random(n))+1.5
art = np.array([r*np.cos(theta), r*np.sin(theta)]).T

theta = np.random.random(n)*np.pi*2
#r = np.sqrt(np.random.random(n)) * (1.5-radiusFAZ_ICP) + radiusFAZ_ICP
r = np.sqrt(np.random.random(n))+1.5
vei = np.array([r*np.cos(theta), r*np.sin(theta)]).T

ICP.add_nodes_from((f'a{i}', {'position':art[i], 'type':'art'}) for i in range(art.shape[0]))
ICP.add_nodes_from((f'v{i}', {'position':vei[i], 'type':'vei'}) for i in range(vei.shape[0]))

PlotGraphICP(ICP)

In [116]:
availableConnections = [np.append(data['position'],n) for n,data in ICP.nodes.data(True) if data['type']=='cap']
for n,pos in ((n,data['position']) for n,data in ICP.nodes.data(True) if data['type']!='cap'):
    closestCapId = np.argmin(np.linalg.norm(pos-np.array(availableConnections)[:,:2], axis=1))
    closestCap = availableConnections.pop(closestCapId)[-1]

    ICP.add_edge(n, closestCap)
    
PlotGraphICP(ICP)

In [117]:
## Create a physiological path 
G = nx.DiGraph(nx.convert_node_labels_to_integers(ICP)) # Copy the graph

artNodes = [n for n,t in G.nodes.data('type') if t=='art']
capNodes = [n for n,t in G.nodes.data('type') if t=='cap']
veiNodes = [n for n,t in G.nodes.data('type') if t=='vei']

newLabelsForDAG = {n:k for k,n in enumerate([*artNodes, *capNodes, *veiNodes])}
G = nx.relabel_nodes(G, newLabelsForDAG)
orientedEdges = [(n1,n2,d) if n1<n2 else (n2,n1,d) for n1,n2,d in G.edges(data=True)]
G.clear_edges()
G.add_edges_from(orientedEdges)

PlotGraphICP(G)

In [118]:
## Remove edges where nodes have in or out degree > 3
## (vessels trifurcate a most). Next step might add 
## connections so we need all nodes with at most 2
## neighbors in each direction.
print("Number of edges before trimming: ", G.number_of_edges())
capNodes = [n for n,t in G.nodes.data('type') if t=='cap']
for n in capNodes:
    while G.out_degree(n)>2:
        G.remove_edge(n, np.random.choice(list(G.successors(n))))
    while G.in_degree(n)>2:
        G.remove_edge(np.random.choice(list(G.predecessors(n))), n)
print("Number of edges after trimming: ", G.number_of_edges())
print(f'{nx.is_directed_acyclic_graph(G)=}')

Number of edges before trimming:  818
Number of edges after trimming:  375
nx.is_directed_acyclic_graph(G)=True


In [119]:
## Connect capillaries with in or out degree zero
for n in capNodes:
    if G.out_degree(n)==0:
        # Potential links that don't add cycles are 
        # nodes after the current node
        # in the topological ordering. 
        # Note this assumes the nodes are labelled
        # in topological order.
        locs = np.array([np.append(pos, node) 
                         for node, pos in G.nodes.data('position') 
                         if (node>n and 
                             (G.in_degree(node)<3 
                              or G.nodes[node]['type']=='vei'))])
        candidateId = locs[np.argmin( # Pick the closest one
            np.linalg.norm(G.nodes[n]['position']-locs[:,:2],axis=1)), -1]
        G.add_edge(n, int(candidateId))

    if G.in_degree(n)==0:
        # Potential links are nodes that appear 
        # before n in the topological ordering.
        locs = np.array([np.append(pos, node) 
                         for node, pos in G.nodes.data('position') 
                         if (node<n and 
                             (G.out_degree(node)<3 
                              or G.nodes[node]['type']=='art'))])
        candidateId = locs[np.argmin( # Pick the closest one
            np.linalg.norm(G.nodes[n]['position']-locs[:,:2],axis=1)), -1]
        G.add_edge(int(candidateId), n)
print(f'{nx.is_directed_acyclic_graph(G)=}')

nx.is_directed_acyclic_graph(G)=True


In [120]:
## Previous step may have left veins or arteries dangling
artNodes = [n for n,t in G.nodes.data('type') if t=='art']
capNodes = [n for n,t in G.nodes.data('type') if t=='cap']
veiNodes = [n for n,t in G.nodes.data('type') if t=='vei']

for n in artNodes:
    if G.out_degree(n)==0:
        locs = np.array([np.append(pos, node)
                         for node,pos in G.nodes.data('position')
                         if (node in capNodes 
                             and G.in_degree(node)<3)])
        candidateId = locs[np.argmin( # Pick the closest one
            np.linalg.norm(G.nodes[n]['position']-locs[:,:2],axis=1)), -1]
        G.add_edge(n, int(candidateId))

for n in veiNodes:
    if G.in_degree(n)==0:
        locs = np.array([np.append(pos, node)
                         for node,pos in G.nodes.data('position')
                         if (node in capNodes 
                             and G.out_degree(node)<3)])
        candidateId = locs[np.argmin( # Pick the closest one
            np.linalg.norm(G.nodes[n]['position']-locs[:,:2],axis=1)), -1]
        G.add_edge(int(candidateId), n)

print(f'{nx.is_directed_acyclic_graph(G)=}')

nx.is_directed_acyclic_graph(G)=True


In [121]:
## Count furcations of capillaries
print(G.number_of_edges(), G.number_of_nodes())

furcOut = {0:0, 1:0, 2:0, 3:0, 4:0}
furcIn = {0:0, 1:0, 2:0, 3:0, 4:0}
for n in (n for n,t in G.nodes.data('type') if t=='cap'):
    outdeg = G.out_degree(n)
    k = min(outdeg, 4)
    furcOut[k]+=1

    indeg = G.in_degree(n)
    k = min(indeg, 4)
    furcIn[k]+=1

print(furcIn, furcOut)
PlotGraphICP(G)

print(f'{nx.is_directed_acyclic_graph(G)=}')

481 310
{0: 0, 1: 151, 2: 103, 3: 36, 4: 0} {0: 0, 1: 144, 2: 113, 3: 33, 4: 0}
nx.is_directed_acyclic_graph(G)=True


## Link SVP and ICP

In [130]:
nx.set_node_attributes(G, {n: {'position':np.append(pos[:2], depth)} for n,pos in G.nodes.data('position')})
GSVP = CreateGraph('../Results-8-patients/Patient1/sim_1.cco', 'mm')
print(f'{nx.is_directed_acyclic_graph(GSVP)=}')

lengthConversion=10.0
The tree has 4015 vessels.
nx.is_directed_acyclic_graph(GSVP)=True


In [131]:
def FindClosestNode(graph, nodePosition, 
                    condition=lambda graph,n: True):
    '''
    Returns the node in graph that is closest to 
    nodePosition and satisfy condition.
    '''
    positions = np.array([np.append(pos, node) 
                          for node,pos in graph.nodes.data('position')
                          if condition(graph, node)])
    return positions[np.argmin(
        np.linalg.norm(positions[:,:-1]-nodePosition, axis=1)),-1]

# Example use:
# FindClosestNode(GSVP, np.zeros(3), lambda g,n: g.out_degree(n)<3)


### Arterial SVP tree to ICP 

In [132]:
newGraph = nx.DiGraph(GSVP)
newGraph.add_nodes_from([(f'ICP{n}', {'position':G.nodes[n]['position'], 'type':G.nodes[n]['type']}) for n in G.nodes])
newGraph.add_edges_from([(f'ICP{n1}', f'ICP{n2}') for n1,n2 in G.edges])
for n in artNodes:
    closest = FindClosestNode(GSVP, G.nodes[n]['position'])
    newGraph.add_edge(closest, f'ICP{n}')

print(f'{nx.is_directed_acyclic_graph(newGraph)=}')

nx.is_directed_acyclic_graph(newGraph)=True


### Venous SVP tree to ICP

In [133]:
GSVP = CreateGraph('../Results-8-patients/Patient2/sim_1.cco', 'mm').reverse()

for n in veiNodes:
    closest = FindClosestNode(GSVP, G.nodes[n]['position'])
    newGraph.add_edge(f'ICP{n}', closest)

print(f'{nx.is_directed_acyclic_graph(newGraph)=}')

lengthConversion=10.0
The tree has 3573 vessels.
nx.is_directed_acyclic_graph(newGraph)=True


In [134]:
%matplotlib qt


pos = {n:pos for n,pos in newGraph.nodes.data('position')}
node_xyz = np.array([pos[n] for n in nx.topological_sort(newGraph)])
edge_xyz = np.array([(pos[u], pos[v]) for u,v in newGraph.edges()])


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Plot the nodes - alpha is scaled by "depth" automatically
ax.scatter(*node_xyz.T, s=1, ec="w")

# Plot the edges
for vizedge in edge_xyz:
    ax.plot(*vizedge.T, color="tab:gray")

  
def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(False)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")


_format_axes(ax)
#ax.set_zscale('log')
fig.tight_layout()
plt.show()
  

In [135]:
def DAGToCCO(G, CCOFileName):
    if not nx.is_directed_acyclic_graph(G):
        print ("Graph is not directed acyclic. Aborting.")
        return 
    
    vesselLine = lambda i,v: (f"{i} "
                            " ".join(str(xi) for x in G.nodes[v[0]]['position']) # Proximal end
                            " "
                            " ".join(str(xi) for x in G.nodes[v[0]]['position']) # Distal end
                            " "
                            " ".join('0.00' for _ in range(5))
                            f"{G[v[0]][v[1]]['radius']}" # Radius
                            " "
                            "0.00" # Flow
                            " "
                            " ".join('0.00' for _ in range(5))
                            " "
                            f"{0}") # Stage

    # We need to store that ordering for the connectivity
    edges = {i:e for i,e in enumerate(G.edges)} 

    connectivityLine = lambda i: f"{i} {n}"

    with open(CCOFileName, 'w') as f:

        f.write("*Tree\n")
        f.write("Tree info blablabla\n\n")

        f.write("*Vessels\n")
        f.write(f"{G.number_of_edges()}\n")
        f.write("\n".join(vesselLine(i,v) for i,v in edges.items()))
        f.write("\n\n")

        f.write("*Connectivity\n")
        f.write("\n".join())



0.0
-0.041
